# Python Wrapper for CMR
`A python library to interface with CMR - Token Demo`

This demo will show how to request an EDL token from CMR while inside a notebook.

## Loading the library 
To use this library, it needs to be loaded into Python. To do this, do one of the following, but not both.

### Choice A) With PIP
Load the library via pip in one of these ways:

* local build with: `./runme.sh -u -p -i`
    * this does an uninstall if you already have it,
    * then packages up a 'wheel' file,
    * and finally install the 'wheel' file using pip3
* lattest from the web: `pip3 install https://github.com/nasa/eo-metadata-tools/releases/download/latest-master/eo_metadata_tools_cmr-0.0.1-py3-none-any.whl`
    * install pre-packaged version from github

### OR Choice B) Reference Dev Copy
This is normally only done if you wish to make local changes or you are testing code. In this case, the command `git clone https://github.com/nasa/eo-metadata-tools` was called in `~/src/project` directory, you may need to change the path below depending on where you performed the clone.

In [ ]:
import os
import sys
sys.path.append(os.path.expanduser('~/src/project/eo-metadata-tools/CMR/python/'))

### 'Import' the library into the notebook
This should be all you need after we get PIP support. Take care to make sure you install to the same version of python if you have multiple instances. If any errors are shown then you may not have installed the library correctly.

In [ ]:
import cmr.auth.token as t
import cmr.search.collection as coll

If something goes wrong, try running the next block to help debug the issue.

In [ ]:
import platform
print ("Python version is {}.".format(platform.python_version()))

import cmr
print ("What build information exists for the library:\n{}".format(cmr.BUILD))

## Printing Tokens - Demo Safety
When working with tokens, you don't want to print them out on the screen, but you might want to look at the value to confirm which token your working with. Use the built in utility function in the utility package to mask out these values for printing to a notebook. This is very importent when sharing your screen.

In [ ]:
import cmr.util.common as com

print ("examples:")
print (com.mask_string("012345678909876543210"))
print(com.mask_dictionary({"token": "012345678909876543210"}, "token"))

# Using a token file


## Shortest Example
Here is the shortest, one line example of doing a search with a saved bearer token. Grab a UAT token from [UAT URS user generated token page](https://uat.urs.earthdata.nasa.gov/user_tokens) and save it to a file called ~/.cmr_token.uat.

In [ ]:
coll.search({'keyword':'modis'},
            filters=[coll.collection_core_fields],
            config=t.use_bearer_token(config={'env': 'uat'}))

## More details on using a token file
In this example we are going to store our token to a file, listed below is how you can specify the file, however the setting is actually the same as the default file if no setting is given.

To get a token, generate one using the [URS user generated token page](https://urs.earthdata.nasa.gov/user_tokens). Take that token and store it in the file `~/.cmr_token`.

In [ ]:
config = {} #use no overrides
print("token: '%s'" % com.mask_string(t.token(t.token_file, config)))

config = {"cmr.token.file": "~/.cmr_token"} #this is the same as the default
print("token: '%s'" % com.mask_string(t.token(t.token_file, config)))

config = {"cmr.token.file": "~/.cmr_token_fake_file"} #this is not the default
print("token: '%s'" % com.mask_string(t.token(t.token_file, config)))

## Using Keychain on Mac OS X
in this example I am using an already existing password saved securly in keychain. For directions on how to setup a keychain record, see the [README](https://github.com/nasa/eo-metadata-tools/tree/master/CMR/python). Keychain may require a human to type in the password, I have clicked "Allways allow" so we may not see it. When using this method, delete the value in `~/.cmr_token` as having a clear text token defeats the security offered by Keychain. 

In [ ]:
options = {'token.manager.service': 'cmr lib token'} #this is not the default
print(com.mask_string(t.token(t.token_manager, options)))

### Search both at once

In [ ]:
options = {"cmr.token.file": "~/.cmr_token_fake_file", 'token.manager.service': 'cmr lib token'}
print(com.mask_string(t.token([t.token_manager, t.token_file], options)))

## Now search with a token
Do a very basic search in production using a token. Results **may** very based on the permisions of the user for which the token is for. This example will use the function called "use_bearer_token" which will create a new configuration dictionary with your token information added to it. Your new configuration can be copied from an existing configuration dictionary if you have one. If saving your token to a file, nothing needs else needs to happen.

`new_config = t.use_bearer_token(config=old_config)`

This example will also show how to turn on logging to get more details out of the internal code.


In [ ]:
# Common settings for both queries
params={'keyword':'modis', 'sort_key':'instrument'} #something to search for
filters=[coll.collection_core_fields] #reduce the values displayed
configs1={} # use all defaults

# For the second query a token will be called from keychain and sent as a Bearer token
configs2=t.use_bearer_token(config=configs1)
coll.set_logging_to("INFO") #logging on

print(coll.search(params, filters=filters, config=configs1, limit=2))
print('\nvs\n')
print(com.mask_string(configs2['authorization']))
print(coll.search(params, filters=filters, config=configs2, limit=2))
coll.set_logging_to("ERROR") #effectivly, logging off

## Bulit in help
I can't remember anything, so here is some built in help which pulls from the python docstring for each function in the token library. The parameter filters out functions, remove it to see all.

In [ ]:
print(t.help_text('token_'))

----
The End